In [16]:
pip install stable-baselines3[extra]


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
pip install gym

^C



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [23]:
import gym
from gym import spaces
import cv2
import numpy as np
import pyautogui
import time


class GeometryDashEnv(gym.Env):
    def __init__(self):
        super(GeometryDashEnv, self).__init__()
        # Define el espacio de observación (estado del juego)
        self.observation_space = spaces.Box(low=0, high=255, shape=(160, 120, 3), dtype=np.uint8)
        
        # Define el espacio de acciones
        self.action_space = spaces.Discrete(2)  # 0: do nothing, 1: jump
        
        # Inicializa el porcentaje de verde (para detectar si el cubo está vivo)
        self.green_per = 0.1
    

    def capture_screen(self, region=(0, 0, 640, 480), resize_dim=(120, 160)):
        screen = pyautogui.screenshot(region=region)
        frame = np.array(screen, dtype=np.uint8)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Convertir a HSV y encontrar el porcentaje de verde
        frame_hsv = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2HSV)
        lower_green = np.array([50, 100, 128])
        upper_green = np.array([70, 255, 255])
        mask = cv2.inRange(frame_hsv, lower_green, upper_green)
        self.green_per = np.sum(mask == 255) / mask.size * 100
        
        # Redimensionar imagen
        frame_resized = cv2.resize(frame_rgb, resize_dim)
        return frame_resized

    def reset(self):
        print("Episodio terminado")
        time.sleep(1)
        return self.capture_screen()

    def step(self, action):
        # Realizar acción
        if action == 1:
            pyautogui.click()
        
        # Capturar el nuevo estado
        self.state = self.capture_screen()
        
        done = False
        reward = 1  # Recompensa por seguir vivo
        
        # Si el porcentaje de verde es muy bajo, podría indicar que el cubo ha muerto
        if self.green_per == 0: 
            done = True
            reward = -50   # Penalización por morir
        
        info = {'green_percentage': self.green_per}
        return self.state, reward, done, info

    def render(self, mode='human'):
        display_img = self.state.copy()  # Haz una copia para no modificar el estado original
        
        if mode == 'human':
            # Dibuja texto con el porcentaje de verde detectado
            cv2.putText(display_img, f'Green %: {self.green_per:.2f}', (10, 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            cv2.imshow('Gameplay with Stats', display_img)
            cv2.waitKey(1)
        elif mode == 'rgb_array':
            return display_img
    def seed(self, seed=None):
        pass




In [24]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# tiempo para abrir la partida

time.sleep(5)
# Suponiendo que 'GeometryDashEnv' es un entorno Gym personalizado para Geometry Dash
env = make_vec_env(GeometryDashEnv, n_envs=1)

model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 2048,
    batch_size = 128,
    n_epochs = 6,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)
model.learn(total_timesteps=10000)

# Guardar y cargar el modelo
model.save("ppo_geometrydash")
del model  # elimina el modelo para demostrar cómo cargarlo
model = PPO.load("ppo_geometrydash")

# Usar el modelo entrenado
obs = env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)
    env.render()


e:\AI_FOR_Geometry_dash\.venv_v2\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cuda device
Wrapping the env in a VecTransposeImage.
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio t

e:\AI_FOR_Geometry_dash\.venv_v2\lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
Episodio terminado
